In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [2]:
True_news = pd.read_csv('True.csv')
Fake_news = pd.read_csv('Fake.csv')

In [3]:
True_news['label'] = 0

In [4]:
Fake_news['label'] = 1

In [5]:
True_news.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [6]:
Fake_news.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [7]:
dataset1 = True_news[['text','label']]
dataset2 = Fake_news[['text','label']]

In [8]:
dataset = pd.concat([dataset1 , dataset2])

In [9]:
dataset.shape

(44898, 2)

In [10]:
dataset.isnull().sum()

text     0
label    0
dtype: int64

In [11]:
dataset['label'].value_counts()

1    23481
0    21417
Name: label, dtype: int64

In [12]:
dataset1.shape # true news

(21417, 2)

In [13]:
dataset2.shape # fake news

(23481, 2)

In [14]:
dataset = dataset.sample(frac = 1)

In [15]:
dataset.head(20)

,text,label
15868,MOSCOW (Reuters) - Kremlin spokesman Dmitry Pe...,0
16141,MADRID (Reuters) - Spain s state prosecutor on...,0
4104,WASHINGTON (Reuters) - President Donald Trump ...,0
4583,(Reuters) - The chairman of the U.S. Senate Ba...,0
736,BEIJING (Reuters) - U.S. President Donald Trum...,0
12361,ISTANBUL (Reuters) - Turkish police summoned a...,0
21413,"LONDON (Reuters) - LexisNexis, a provider of l...",0
15637,The main stream media has done a great job of ...,1
467,Donald Trump is a horrible president. Everyone...,1
8214,A Los Angeles County sheriff s deputy accident...,1


In [16]:
ps = WordNetLemmatizer()

In [17]:
stop_words = nltk.corpus.stopwords.words('english')

In [18]:
nltk.download('wordnet')

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [19]:
def cleaning_data(row):
    
    # convert text to into lower case
    row = row.lower() 
    
    # this line of code only take words from text and remove number and special character using RegX
    row = re.sub('[^a-zA-Z]' , ' ' , row)
    
    # split the data and make token.
    token = row.split() 
    
    # lemmatize the word and remove stop words like a, an , the , is ,are ...
    news = [ps.lemmatize(word) for word in token if not word in stop_words]  
    
    # finaly join all the token with space
    cleanned_news = ' '.join(news) 
    
    # return cleanned data
    return cleanned_news

In [20]:
dataset['text'] = dataset['text'].apply(lambda x : cleaning_data(x))

In [21]:
dataset.isnull().sum()

text     0
label    0
dtype: int64

In [22]:
vectorizer = TfidfVectorizer(max_features = 50000 , lowercase=False , ngram_range=(1,2))

In [23]:
dataset.shape

(44898, 2)

In [24]:
X = dataset.iloc[:35000,0]
y = dataset.iloc[:35000,1]

In [25]:
X.head()

15868    moscow reuters kremlin spokesman dmitry peskov...
16141    madrid reuters spain state prosecutor monday c...
4104     washington reuters president donald trump comp...
4583     reuters chairman u senate banking committee sa...
736      beijing reuters u president donald trump told ...
Name: text, dtype: object

In [26]:
y.head()

15868    0
16141    0
4104     0
4583     0
736      0
Name: label, dtype: int64

In [27]:
from sklearn.model_selection import train_test_split
train_data , test_data , train_label , test_label = train_test_split(X , y , test_size = 0.2 ,random_state = 0)

In [28]:
vec_train_data = vectorizer.fit_transform(train_data)

In [29]:
vec_train_data = vec_train_data.toarray()

In [30]:
train_data.shape , test_data.shape

((28000,), (7000,))

In [31]:
vec_test_data = vectorizer.transform(test_data).toarray()

In [32]:
vec_train_data.shape , vec_test_data.shape

((28000, 50000), (7000, 50000))

In [33]:
train_label.value_counts() # balanced partition

1    14615
0    13385
Name: label, dtype: int64

In [34]:
test_label.value_counts() # balanced partition

1    3692
0    3308
Name: label, dtype: int64

In [35]:
training_data = pd.DataFrame(vec_train_data , columns=vectorizer.get_feature_names())
testing_data = pd.DataFrame(vec_test_data , columns= vectorizer.get_feature_names())

In [36]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,classification_report

In [37]:
clf = MultinomialNB()

In [38]:
clf.fit(training_data, train_label)
y_pred  = clf.predict(testing_data)

In [39]:
pd.Series(y_pred).value_counts()

1    3726
0    3274
dtype: int64

In [40]:
test_label.value_counts()

1    3692
0    3308
Name: label, dtype: int64

In [41]:
print(classification_report(test_label , y_pred))

              precision    recall  f1-score   support

           0       0.95      0.94      0.95      3308
           1       0.95      0.96      0.95      3692

    accuracy                           0.95      7000
   macro avg       0.95      0.95      0.95      7000
weighted avg       0.95      0.95      0.95      7000



In [42]:
y_pred_train = clf.predict(training_data)
print(classification_report(train_label , y_pred_train))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96     13385
           1       0.96      0.96      0.96     14615

    accuracy                           0.96     28000
   macro avg       0.96      0.96      0.96     28000
weighted avg       0.96      0.96      0.96     28000



In [43]:
accuracy_score(train_label , y_pred_train)

0.9595357142857143

In [44]:
accuracy_score(test_label , y_pred)

0.9514285714285714

In [52]:
confusion_matrix(y_pred_train,train_label)

array([[12798,   546],
       [  587, 14069]], dtype=int64)

In [45]:
news = cleaning_data(str("Imposters posing as army personnel on the social media have been called out by the Indian Army as false news and disinformation."))

In [46]:
single_prediction = clf.predict(vectorizer.transform([news]).toarray())
single_prediction

array([1], dtype=int64)

In [47]:
import joblib 

In [48]:
joblib.dump(clf , 'model.pkl')

['model.pkl']

In [49]:
model = joblib.load('model.pkl')